In [1]:
import sys
music_gen_path = "/home/henri/Documents/Git/MusicGenerationWithML"
if music_gen_path not in sys.path:
    sys.path.append(music_gen_path)
import numpy as np
from util.music_theory import *
from util.sixteenth_array import *
from util.music_parsing import *
#import file_management import midi_index
from file_management import midi_index
from music_generation.random_generate import *

hello = randomlyGenerateBar2()
hi = hello.toSong()

Directory changed to /home/henri/Documents/Git/MusicGenerationWithML
File data/MUSICFILEINDEX.csv exists


In [2]:
hi.printMe()


bpm: 120
time_sig: [4, 4]
key_sig: C
instrument: 1
ppqn: 480
pulse_len: 1920
bar_len: 1920
melody: [62, 60, 53, 55, 57, 59, 60]
melody_time_intervals: [[0, 240], [240, 480], [480, 720], [720, 1200], [1200, 1440], [1440, 1680], [1680, 1920]]
chords: [<util.music_theory.Harmony instance at 0x2e00950>]
chord_time_intervals: [[0, 1920]]


In [3]:
ola = hi.toSixteenthArray()

In [4]:
#ola.melody_arr

In [3]:
music_path = 'data/midi_my_data/Music_AI_Data.mid'
song = Song()
song.loadFromMidi(music_path)
arr = song.toSixteenthArray()
splitted = arr.splitByBarRest()


In [10]:
song.printMe()

bpm: 120
time_sig: [4, 4]
key_sig: G
instrument: 1
ppqn: 480
pulse_len: 416640
bar_len: 1920
melody: [63, 75, 70, 68, 63, 68, 66, 51, 54, 51, 75, 70, 66, 61, 56, 49, 51, 54, 58, 51, 54, 51, 75, 70, 66, 61, 56, 49, 51, 54, 58, 75, 75, 73, 73, 71, 71, 70, 70, 68, 68, 66, 66, 65, 65, 63, 63, 71, 70, 66, 71, 70, 66, 64, 64, 63, 64, 64, 63, 66, 66, 58, 59, 58, 54, 58, 59, 58, 54, 68, 71, 71, 71, 78, 75, 73, 71, 75, 78, 78, 75, 73, 71, 68, 71, 75, 73, 71, 71, 68, 75, 73, 71, 75, 78, 75, 73, 71, 73, 75, 73, 71, 71, 68, 75, 73, 71, 71, 68, 75, 73, 71, 75, 78, 75, 73, 71, 73, 75, 73, 71, 71, 73, 83, 80, 78, 80, 78, 75, 73, 71, 73, 83, 80, 83, 80, 78, 75, 73, 71, 75, 78, 75, 73, 71, 73, 83, 80, 78, 80, 78, 75, 73, 71, 73, 83, 80, 83, 80, 78, 75, 73, 71, 75, 78, 75, 73, 71, 73, 77, 76, 67, 69, 72, 74, 77, 76, 67, 69, 72, 74, 62, 65, 62, 62, 67, 62, 60, 62, 69, 62, 62, 70, 69, 65, 62, 69, 74, 62, 60, 60, 57, 64, 62, 64, 64, 64, 71, 69, 67, 66, 64, 64, 64, 71, 73, 69, 71, 64, 71, 71, 71, 71, 69, 69

In [7]:
for melody, chord in zip(
    np.array(arr.melody_arr).reshape((-1, 16)),
    np.array(arr.chords_arr).reshape((-1, 16))):
    
    print melody, len(chord)
    print " ", chord, len(chord)
    print ""

[63 -1 75 -1 70 -1 68 -1 -1 -1 63 -1 68 -1 66 -1] 16
  [<util.music_theory.Harmony instance at 0x2418a28> -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1] 16

[-2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2] 16
  [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1] 16

[51 -1 -1 54 -1 -1 51 75 70 66 61 56 49 51 54 58] 16
  [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1] 16

[51 -1 -1 54 -1 -1 51 75 70 66 61 56 49 51 54 58] 16
  [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1] 16

[-2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2] 16
  [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1] 16

[75 75 73 73 71 71 70 70 68 68 66 66 65 65 63 63] 16
  [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1] 16

[-2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2] 16
  [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1] 16

[-2 -2 71 -1 70 -1 66 -1 -1 -1 71 -1 70 -1 66 -1] 16
  [<util.music_theory.Harmony instance at 0x2418b90> -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1] 16

[-1 -1 64 -1 64 -1 63 -1 64 -1 -

In [4]:
song.toMidi('test.mid')
song.printMe()
#song2 = Song()
#song2.loadFromMidi('test.mid')
#song2.printMe()

bpm: 120
time_sig: [4, 4]
key_sig: G
instrument: 1
ppqn: 480
pulse_len: 416640
bar_len: 1920
melody: [63, 75, 70, 68, 63, 68, 66, 51, 54, 51, 75, 70, 66, 61, 56, 49, 51, 54, 58, 51, 54, 51, 75, 70, 66, 61, 56, 49, 51, 54, 58, 75, 75, 73, 73, 71, 71, 70, 70, 68, 68, 66, 66, 65, 65, 63, 63, 71, 70, 66, 71, 70, 66, 64, 64, 63, 64, 64, 63, 66, 66, 58, 59, 58, 54, 58, 59, 58, 54, 68, 71, 71, 71, 78, 75, 73, 71, 75, 78, 78, 75, 73, 71, 68, 71, 75, 73, 71, 71, 68, 75, 73, 71, 75, 78, 75, 73, 71, 73, 75, 73, 71, 71, 68, 75, 73, 71, 71, 68, 75, 73, 71, 75, 78, 75, 73, 71, 73, 75, 73, 71, 71, 73, 83, 80, 78, 80, 78, 75, 73, 71, 73, 83, 80, 83, 80, 78, 75, 73, 71, 75, 78, 75, 73, 71, 73, 83, 80, 78, 80, 78, 75, 73, 71, 73, 83, 80, 83, 80, 78, 75, 73, 71, 75, 78, 75, 73, 71, 73, 77, 76, 67, 69, 72, 74, 77, 76, 67, 69, 72, 74, 62, 65, 62, 62, 67, 62, 60, 62, 69, 62, 62, 70, 69, 65, 62, 69, 74, 62, 60, 60, 57, 64, 62, 64, 64, 64, 71, 69, 67, 66, 64, 64, 64, 71, 73, 69, 71, 64, 71, 71, 71, 71, 69, 69

In [ ]:
song.printMe()

In [ ]:
arr.toSong().printMe()

In [ ]:
arr.melody_arr

In [24]:
# Check Greater than
def chGt(x, y, error_type):
    if x <= y:
        message = "{} error: {} <= {}".format(error_type, x, y)
        raise Exception(message)

def splitByBarRest(self):
    melody_arr = self.melody_arr
    melody_arr_len = len(self.melody_arr)
    start = 0
    end = SIXTEENTH_BAR_LEN

    result = []

    while True:
        # ignore rest
        while end <= melody_arr_len and np.all(np.array(melody_arr[start:end]) == REST):
            start += SIXTEENTH_BAR_LEN
            end += SIXTEENTH_BAR_LEN

        if end > melody_arr_len:
            return result

        #print melody_arr[start:end], np.any(np.array(melody_arr[start:end]) != REST)
        
        # find maximal streak of consecutive non-empty bars
        while (end <= melody_arr_len 
               and np.any(np.array(melody_arr[end - SIXTEENTH_BAR_LEN:end]) != REST)):
            end += SIXTEENTH_BAR_LEN
        end -= SIXTEENTH_BAR_LEN
        
        chGt(end, start, "splitByBarRest")
        
        #print melody_arr[end - SIXTEENTH_BAR_LEN:end]

        new_sixteenth_arr = SixteenthArray()
        new_sixteenth_arr.loadFromArguments(
            self.melody_arr[start:end], 
            self.chords_arr[start:end], 
            self.key_sig, self.bpm, self.time_sig)
        result.append(new_sixteenth_arr)
        
        print start, end
        #print " ",  melody_arr[start:end]
        #print " ",  melody_arr[end:end + SIXTEENTH_BAR_LEN]
        #print " ",  melody_arr[end + SIXTEENTH_BAR_LEN:end + 2*SIXTEENTH_BAR_LEN]

        start = end
        end = start + SIXTEENTH_BAR_LEN

splitted = splitByBarRest(arr)

0 16
32 64
80 160
176 256
272 416
432 512
528 1408
1424 1488
1504 1552
1568 1840
1856 2640
2656 3200


In [8]:


def myMidiNumToNote(x):
    if x >= 0:
        return midiNumToNote(x)
    else:
        return x

print len(splitted)
for yo in splitted:
    #yo.toSong().printMe()
    print yo.n_bars
    for row in np.array(yo.melody_arr).reshape((-1, 16)):
        print map(myMidiNumToNote, row)

36
1
['D#', -1, 'D#', -1, 'A#', -1, 'G#', -1, -1, -1, 'D#', -1, 'G#', -1, 'F#', -1]
2
['D#', -1, -1, 'F#', -1, -1, 'D#', 'D#', 'A#', 'F#', 'C#', 'G#', 'C#', 'D#', 'F#', 'A#']
['D#', -1, -1, 'F#', -1, -1, 'D#', 'D#', 'A#', 'F#', 'C#', 'G#', 'C#', 'D#', 'F#', 'A#']
1
['D#', 'D#', 'C#', 'C#', 'B', 'B', 'A#', 'A#', 'G#', 'G#', 'F#', 'F#', 'F', 'F', 'D#', 'D#']
4
[-2, -2, 'B', -1, 'A#', -1, 'F#', -1, -1, -1, 'B', -1, 'A#', -1, 'F#', -1]
[-1, -1, 'E', -1, 'E', -1, 'D#', -1, 'E', -1, -1, -1, 'E', -1, -1, -1]
['D#', -1, -1, -1, 'F#', -1, 'F#', -1, -1, -1, -1, -1, -1, -1, -1, -1]
['A#', -1, 'B', -1, 'A#', -1, 'F#', -1, 'A#', -1, 'B', -1, 'A#', -1, 'F#', -1]
5
[-2, -2, -2, -2, -2, -2, -2, -2, -2, -2, 'G#', -1, 'B', 'B', -1, -1]
['B', -1, -2, -2, -2, -2, -2, -2, -2, -2, 'F#', -1, 'D#', 'C#', 'B', -1]
['D#', -1, 'F#', -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
['F#', -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 'D#', 'C#', 'B', -1]
['G#', -1, 'B', -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1

In [10]:
#midi_index.update()

In [4]:
np.random.seed(0)

In [5]:
l = randomlyGenerateBar2().melody_arr
print l
map(midiNumToNote, filter(lambda x: x >= 0, l))

[57, -1, 53, -1, 55, -1, -1, -1, 53, -1, 59, -1, 62, -1, 60, -1]


['A', 'F', 'G', 'F', 'B', 'D', 'C']

TypeError: sixteenthToTimeIntervalFormat() takes exactly 2 arguments (1 given)

In [11]:
mid = MidiFile(music_path)

def parseTrack(track):
    seq = NoteSequence(480)
    acc_time = 0
    note_starts = {}

    for msg in track:
        print msg
        acc_time += msg.time
        continue
        if msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity==0):
            # (note_off is equivalent to note_on with 0 velocity)
            # (note that midi files can play notes then leave a brief rest before the next note
            #  which isn't explicitly marked as a rest)
            print msg.velocity, msg.note, msg.time

            chIn(msg.note, note_starts.keys(), "note_off")

            print 
            seq.addNote(msg.note, note_starts[msg.note], acc_time)

            del note_starts[msg.note]


        elif msg.type == 'note_on':
            print msg.velocity, msg.note, msg.time
            # default: velocity 64, otherwise 0
            # note: between 0 and 127 inclusively where 60 is middle C (40th note on piano),
            #       A0 is 21, and C8 is 108
            # time: specifies how much you wait before playing this note next
            #       eg. time=0 means you play it immediately as you get the message
            #       the units are ticks

            chNotIn(msg.note, note_starts.keys(), "note_on")
            note_starts[msg.note] = acc_time
            
        elif msg.type == 'key_signature':
            print msg.key
            # can be 'Dbm' for D flat minor
            # default 'C' for C major

        elif msg.type == 'time_signature':
            print msg.numerator, msg.denominator, msg.clocks_per_click
            #print msg.notated_32nd_notes_per_beat
            # default 4, 4, 24, 8

            # I'm not sure how to handle other clocks_per_click and notated_32nd_notes_per_beat values
            chEq(msg.clocks_per_click, 24, "clocks_per_tick")
            chEq(msg.notated_32nd_notes_per_beat, 8, "notated_32nd_notes_per_beat")

            chEq(msg.numerator, 4, "time_sig numerator")
            chEq(msg.denominator, 4, "time_sig denominator")
            
        elif msg.type == 'set_tempo':
            print msg.tempo
            # possible: 0 ... 16777215
            # default: 500 000
            # this means 500 000 microseconds (0.5 s) per beat (quarter note)
            # thus the default is 120 BMP (beats per minute)

    chEq(len(note_starts.keys()), 0, "note_dictionary_empty")

    return seq

In [12]:
parseTrack(mid.tracks[1])

<meta message key_signature key='Gb' time=0>
<meta message midi_port port=0 time=0>
note_on channel=0 note=51 velocity=80 time=0
note_on channel=0 note=54 velocity=80 time=0
note_on channel=0 note=58 velocity=80 time=0
note_on channel=0 note=51 velocity=0 time=1823
note_on channel=0 note=54 velocity=0 time=0
note_on channel=0 note=58 velocity=0 time=0
<meta message key_signature key='Cb' time=11617>
note_on channel=0 note=44 velocity=80 time=0
note_on channel=0 note=47 velocity=80 time=0
note_on channel=0 note=51 velocity=80 time=0
note_on channel=0 note=44 velocity=0 time=1823
note_on channel=0 note=47 velocity=0 time=0
note_on channel=0 note=51 velocity=0 time=0
note_on channel=0 note=40 velocity=80 time=97
note_on channel=0 note=44 velocity=80 time=0
note_on channel=0 note=47 velocity=80 time=0
note_on channel=0 note=40 velocity=0 time=1823
note_on channel=0 note=44 velocity=0 time=0
note_on channel=0 note=47 velocity=0 time=0
note_on channel=0 note=47 velocity=80 time=97
note_on ch

<util.music_parsing.NoteSequence instance at 0x2420200>

In [30]:
parseTrack(mid.tracks[0])

<meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
<meta message key_signature key='Gb' time=0>
<meta message set_tempo tempo=500000 time=0>
control_change channel=0 control=121 value=0 time=0
program_change channel=0 program=0 time=0
control_change channel=0 control=7 value=100 time=0
control_change channel=0 control=10 value=64 time=0
control_change channel=0 control=91 value=0 time=0
control_change channel=0 control=93 value=0 time=0
<meta message midi_port port=0 time=0>
note_on channel=0 note=63 velocity=80 time=0
note_on channel=0 note=63 velocity=0 time=227
note_on channel=0 note=75 velocity=80 time=13
note_on channel=0 note=75 velocity=0 time=227
note_on channel=0 note=70 velocity=80 time=13
note_on channel=0 note=70 velocity=0 time=227
note_on channel=0 note=68 velocity=80 time=13
note_on channel=0 note=68 velocity=0 time=455
note_on channel=0 note=63 velocity=80 time=25
note_on channel=0 note=63 velocity=0 time=227

<util.music_parsing.NoteSequence instance at 0x3da4b00>

## Old

## Iterate Through Songs

In [28]:
import pandas as pd
df = pd.read_csv('data/MUSICFILEINDEX.csv', header=0)

In [29]:
df.columns.values

array(['entry', 'file1', 'used', 'personal_rating', 'actual_tempo',
       'n_bar', 'melody_type', 'all_key_notes', 'beat_division', 'tonic',
       'quality', 'actual_tonic', 'pick_up_size', 'time signature',
       'song_name', 'artist', 'readable_file'], dtype=object)

In [30]:
used_files = df[df['used'] >= 0]['file1'].values
for f in used_files:
    try:
        song = Song()
        song.loadFromMidi(f)
        #print "Success:", f
    except Exception as ex:
        print "Error:", str(ex)
        print "  ", f

Error: ([55, 62, 64], ['G', 'D', 'E'])
   data/midi_siraj-music/Blank Space - Chorus.midi
Error: ([55, 62, 64], ['G', 'D', 'E'])
   data/midi_siraj-music/Blank Space - Pre Chorus.midi
Error: note_on error: 60 in [65, 60, 57]
   data/midi_siraj-music/Dark Horse - Verse.midi
Error: ([60, 62, 65], ['C', 'D', 'F'])
   data/midi_siraj-music/Viva La Vida - Verse.midi
Error: ([57, 64, 67], ['A', 'E', 'G'])
   data/midi_siraj-music/I Kissed A Girl - Verse.midi
Error: time_sig numerator error: 3 != 4
   data/midi_my_data/MIDI_Test.mid
Error: ([60, 62, 67], ['C', 'D', 'G'])
   data/midi_siraj-music/All The Small Things - Chorus.midi
Error: ([60, 62, 67], ['C', 'D', 'G'])
   data/midi_siraj-music/All The Small Things - Verse.midi
Error: ([59, 62, 68], ['B', 'D', 'G#'])
   data/midi_siraj-music/Baby I'm Yours - Chorus.midi
Error: ([60, 62, 67], ['C', 'D', 'G'])
   data/midi_siraj-music/Barbie Girl - Chorus.midi
Error: ([60, 62, 67], ['C', 'D', 'G'])
   data/midi_siraj-music/Barbie Girl - Verse.mid

In [ ]:
import csv

def csvToMatrix(file_name):
    result = None
    with open(file_name, 'rb') as f:
        reader = csv.reader(f)
        result = list(reader)
    return result

def matrixToCsv(mat, file_name):
    with open(file_name, 'wb') as f:
        writer = csv.writer(f)
        writer.writerows(mat)
        
